# OUI

### package load 및 device 지정

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd

In [2]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [3]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [4]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"] = "3"
#os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
#os.environ["TORCH_USE_CUDA_DSA"] = '1'
#device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
#torch.cuda.device_count()
device = torch.device("cuda:3")
print(device)

cuda:3


In [5]:
emotion_to_idx = {0:'분노', 1:'중립', 2:'당황', 3:'슬픔', 4:'기쁨', 5:'불안', 6:'느긋'}
idx_to_emotion = {"분노": 0, "중립":1, "당황":2, "슬픔":3, "기쁨":4, "불안":5, "느긋":6}

In [6]:
import re
all_df["cat1_idx"] = all_df["cat1"].map(lambda row: idx_to_emotion[row])
all_df["sentence"] = all_df["sentence"].apply(lambda x: re.sub(r'["\']', '', x))
all_df = all_df[["sentence", "cat1_idx"]]
all_df.head(3)

NameError: name 'all_df' is not defined

In [ ]:
batch_size = 64
max_len = 64

### 데이터세트 구성

In [7]:
train_dataset = nlp.data.TSVDataset(os.path.join("../dataset", "train.tsv"), num_discard_samples=1)
test_dataset = nlp.data.TSVDataset(os.path.join("../dataset", "valid.tsv"), num_discard_samples=1)

In [8]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

using cached model. /home/j-j10a506/oui/KoBERT/.cache/kobert_v1.zip
using cached model. /home/j-j10a506/oui/KoBERT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [9]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /home/j-j10a506/oui/KoBERT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [10]:
batch_size = 64
max_len = 100

In [11]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = []
        self.labels = []
        for i in dataset:
            if len(i)!=2:
                continue

            self.sentences.append(transform([i[sent_idx]]))
            self.labels.append(np.int32(i[label_idx]))
        

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [12]:
data_train = BERTDataset(train_dataset, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test_dataset, 0, 1, tok, max_len, True, False)

In [13]:
train_dataloader = DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = DataLoader(data_test, batch_size=batch_size, num_workers=5)

### kobert load
* 하이퍼파라미터 조정
  * dropout 비율
  * learning rate
  * warmup_ratio
  * learning_rate
  * max_grad_norm

In [14]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

In [15]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

### 학습
* best 모델 저장
* epoch 수: 최대 20회

In [16]:
## Setting parameters
warmup_ratio = 0.1 # 학습률 warm up 비율로 학습률을 초기에 점진적으로 높이는데 사용
num_epochs = 20
max_grad_norm = 1 # Gradient clipping에 사용됨 
log_interval = 200
learning_rate =  1e-5

In [17]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [18]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [19]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [20]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [21]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/1110 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.0457327365875244 train acc 0.03125
epoch 1 batch id 201 loss 1.8949710130691528 train acc 0.2605721393034826
epoch 1 batch id 401 loss 1.8288354873657227 train acc 0.27879519950124687
epoch 1 batch id 601 loss 1.866255760192871 train acc 0.2880615640599002
epoch 1 batch id 801 loss 1.502577781677246 train acc 0.2993913857677903
epoch 1 batch id 1001 loss 1.6591631174087524 train acc 0.3123126873126873
epoch 1 train acc 0.32150321277159516


  0%|          | 0/278 [00:00<?, ?it/s]

epoch 1 test acc 0.4506070143884892


  0%|          | 0/1110 [00:00<?, ?it/s]

...

epoch 3 batch id 1 loss 1.4266597032546997 train acc 0.546875
epoch 3 batch id 201 loss 1.1954253911972046 train acc 0.550761815920398
epoch 3 batch id 401 loss 1.2541453838348389 train acc 0.5515897755610972
epoch 3 batch id 601 loss 1.0679904222488403 train acc 0.5563383943427621
epoch 3 batch id 801 loss 1.004834532737732 train acc 0.5601201622971286
epoch 3 batch id 1001 loss 0.9544792175292969 train acc 0.561781968031968
epoch 3 train acc 0.5630580948595655


  0%|          | 0/278 [00:00<?, ?it/s]

epoch 3 test acc 0.5721470323741007


  0%|          | 0/1110 [00:00<?, ?it/s]

epoch 4 batch id 1 loss 1.217409610748291 train acc 0.640625
epoch 4 batch id 201 loss 1.0904052257537842 train acc 0.584110696517413
epoch 4 batch id 401 loss 1.1373180150985718 train acc 0.5836970074812967
epoch 4 batch id 601 loss 1.0073188543319702 train acc 0.586730449251248
epoch 4 batch id 801 loss 0.9366891384124756 train acc 0.589146379525593


In [ ]:
MODEL_PATH = "../myModel"
torch.save(model.state_dict(), os.path.join(MODEL_PATH, "oui_20240325_2.pt"))

In [ ]:
# https://velog.io/@fhflwhwl5/Python-KoBERT-7%EA%B0%80%EC%A7%80-%EA%B0%90%EC%A0%95%EC%9D%98-%EB%8B%A4%EC%A4%91%EA%B0%90%EC%84%B1%EB%B6%84%EB%A5%98%EB%AA%A8%EB%8D%B8-%EA%B5%AC%ED%98%84%ED%95%98%EA%B8%B0